In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


In [10]:
weather_df = pd.read_csv("../WeatherPy/output_data_cities.csv")
weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Petropavlovsk-Kamchatskiy,90,RU,1619894927,100,53.0452,158.6483,32.00,1.74
1,Christchurch,0,NZ,1619894571,100,-43.5333,172.6333,30.20,2.30
2,Vila Franca do Campo,75,PT,1619894635,88,37.7167,-25.4333,60.80,10.36
3,Barrow,90,US,1619894464,86,71.2906,-156.7887,21.20,10.36
4,Los Pozos,100,VE,1619894928,43,8.4518,-62.7268,90.50,11.16
...,...,...,...,...,...,...,...,...,...
533,Buenos Aires,0,AR,1619895070,64,-34.6132,-58.3772,78.80,4.61
534,Duvan,100,RU,1619895075,56,55.6950,57.9024,52.32,17.47
535,Port-Gentil,20,GA,1619895075,79,-0.7193,8.7815,84.20,6.91
536,Kota Kinabalu,20,MY,1619895075,94,5.9749,116.0724,77.00,4.61


In [15]:
# Configure gmaps
gmaps.configure(api_key= g_key)

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_df["Humidity"]

In [16]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
max_intensity

100

In [19]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 80)].dropna()

narrowed_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
14,Port Keats,0,AU,1619894930,58,-14.2500,129.5500,72.86,5.88
18,Salta,0,AR,1619894931,43,-24.7859,-65.4117,73.99,3.44
49,Atuona,0,PF,1619894359,73,-9.8000,-139.0333,78.66,9.46
123,Saint-Philippe,0,RE,1619894520,78,-21.3585,55.7679,75.20,5.75
197,Kiryat Gat,0,IL,1619894284,57,31.6100,34.7642,73.99,3.62
211,Pisco,0,PE,1619894549,73,-13.7000,-76.2167,71.60,9.22
261,Saint-Pierre,0,RE,1619895015,78,-21.3393,55.4781,75.20,5.75
394,Gaza,0,PS,1619895043,46,31.5000,34.4667,73.99,2.30
521,Pinheiro Machado,0,BR,1619895072,58,-31.5783,-53.3811,75.99,3.00
533,Buenos Aires,0,AR,1619895070,64,-34.6132,-58.3772,78.80,4.61


In [20]:
# Create a hotel_df
hotels_df = narrowed_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotels_df["Hotel Name"] = ""

# Display the result
hotels_df

,City,Country,Lat,Lng,Hotel Name
14,Port Keats,AU,-14.2500,129.5500,
18,Salta,AR,-24.7859,-65.4117,
49,Atuona,PF,-9.8000,-139.0333,
123,Saint-Philippe,RE,-21.3585,55.7679,
197,Kiryat Gat,IL,31.6100,34.7642,
211,Pisco,PE,-13.7000,-76.2167,
261,Saint-Pierre,RE,-21.3393,55.4781,
394,Gaza,PS,31.5000,34.4667,
521,Pinheiro Machado,BR,-31.5783,-53.3811,
533,Buenos Aires,AR,-34.6132,-58.3772,


In [22]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [25]:
for index, row in hotels_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"

    print(f"Getting Results Index: {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    
    results = response['results']
    
   
    try:
        print(f"The closest hotel in {city_name} is {results[0]['name']}.")
        hotels_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result.")
        
    print("--------------------------------------------")
    
    



print("-------Complete-------")

Getting Results Index: 14: Port Keats.
The closest hotel in Port Keats is TDC Village (Hundred Man Camp).
--------------------------------------------
Getting Results Index: 18: Salta.
The closest hotel in Salta is Sheraton Salta Hotel.
--------------------------------------------
Getting Results Index: 49: Atuona.
Missing field/result.
--------------------------------------------
Getting Results Index: 123: Saint-Philippe.
The closest hotel in Saint-Philippe is Les Embruns Du Baril.
--------------------------------------------
Getting Results Index: 197: Kiryat Gat.
The closest hotel in Kiryat Gat is Desert Gat.
--------------------------------------------
Getting Results Index: 211: Pisco.
Missing field/result.
--------------------------------------------
Getting Results Index: 261: Saint-Pierre.
The closest hotel in Saint-Pierre is Villa Delisle, Hotel & Spa.
--------------------------------------------
Getting Results Index: 394: Gaza.
Missing field/result.
------------------------

In [26]:
hotels_df

,City,Country,Lat,Lng,Hotel Name
14,Port Keats,AU,-14.2500,129.5500,TDC Village (Hundred Man Camp)
18,Salta,AR,-24.7859,-65.4117,Sheraton Salta Hotel
49,Atuona,PF,-9.8000,-139.0333,
123,Saint-Philippe,RE,-21.3585,55.7679,Les Embruns Du Baril
197,Kiryat Gat,IL,31.6100,34.7642,Desert Gat
211,Pisco,PE,-13.7000,-76.2167,
261,Saint-Pierre,RE,-21.3393,55.4781,"Villa Delisle, Hotel & Spa"
394,Gaza,PS,31.5000,34.4667,
521,Pinheiro Machado,BR,-31.5783,-53.3811,Hotel Chanceller
533,Buenos Aires,AR,-34.6132,-58.3772,Emperador Hotel Buenos Aires


In [27]:
#template from starter code
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotels_df[["Lat", "Lng"]]


In [28]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))